In [6]:
%matplotlib inline

# Getting Started With DESC

DESC is a 3D MHD equilibrium and optimization code suite, which solves the ideal MHD equilibrium equations to find stellarator equilibria. 

Like VMEC, DESC requires 3 main inputs to define the equilibrium problem:

 - Pressure Profile
 - Rotational Transform Profile
 - Last Closed Flux Surface Boundary Shape

DESC can be ran both with a text input file from the command line (e.g. `python -m desc INPUT_FILE`) or through python scripts (which offers more options than the text file for equilibrium solving and optimization). This tutorial notebook will focus on running from the command line.

## Installing DESC

The installation instructions located in the [installation documentation page](https://desc-docs.readthedocs.io/en/latest/installation.html#installation).

Once these instructions are followed and DESC is installed correctly, you should be able to `import desc` without any errors

In [1]:
import desc

## Running DESC On HELIOTRON Example Case

We will now see how to run DESC on the HELIOTRON example case.

We could run this from a DESC input file directly, but we will first show how you can convert a VMEC input file to a DESC input file.

In this directory is a VMEC input file titled `input.HELIOTRON` which contains a finite beta HELIOTRON fixed boundary stellarator. To convert we can simply run DESC from the command line directly on the VMEC input file with the following command: `desc -vv input.HELIOTRON -o SOLOVEV_output.h5` (if pip installed DESC) or `python -m desc -vv input.HELIOTRON -o SOLOVEV_output.h5` The `-vv` flag is just for verbosity, and the `-o` flag allows us to specify the name of the output HDF5 file (the default name is `XXX_output.h5`. See the [command line docs](https://desc-docs.readthedocs.io/en/latest/command_line.html#command-line-interface) for more information.

This will automatically convert the VMEC input file to a DESC input file (named by default `input.HELIOTRON_desc`), and then run DESC with that input file. However, there are many DESC solver options which do not have analogs in VMEC, and so this automatic conversion may not result in a great solution. It is then recommended to modify the solver options to allow for a better convergence to an equilibrium solution. The DESC input file are [documented here](https://desc-docs.readthedocs.io/en/latest/input.html#input-file).

As a general rule, the most important parameters to change are related to the spectral resolution and the continuation method parameters. DESC leverages a multi-grid continuation method to allow for more robust convergence of highly shaped equilibria, and this is controlled by the following parameters:

 **Spectral Resolution**
 
 - ` L_rad` (int): Maximum radial mode number for the Fourier-Zernike basis, . Default = `M_pol` if `spectral_indexing = ANSI`, or 2*M_pol if `spectral_indexing = Fringe`. For more information see [Basis functions and collocation nodes](https://desc-docs.readthedocs.io/en/latest/notebooks/basis_grid.html).
 - `M_pol` (int): Maximum poloidal mode number for the Fourier-Zernike basis, . **Required Input**.
 - `N_tor` (int): Maximum toroidal mode number for the Fourier-Zernike basis, . Default = 0.
 - `L_grid` (int): Radial resolution of nodes in collocation grid. Default = `M_grid` if `spectral_indexing = ANSI`, or `2*M_grid` if `spectral_indexing = Fringe`.
 - `M_grid` (int): Poloidal resolution of nodes in collocation grid. Default = `round(1.5*M_pol)`.
 - `N_grid` (int): Toroidal resolution of nodes in collocation grid. Default = `round(1.5*N_tor)`.

When `M_grid = M_pol` the number of collocation nodes in each toroidal cross-section is equal to the number of Zernike polynomial in the basis set. When `N_grid = N_tor` the number of nodes with unique toroidal angles is equal to the number of terms in the toroidal Fourier series. Convergence is typically superior when the number of nodes exceeds the number of spectral coefficients, but this adds compuational cost.

These arguments can be passed as arrays, where each element denotes the value to use at that iteration. Array elements are deliminated by either a space ` ` , comma `,` , or semi-colon `;`. Arrays can also be created using the shorthand notation `start:interval:end` and `(value)x(repititions)`. For example, an input line of `M_pol = M_pol  =  6:2:10, 10; 11x2 12` is equivalent to `M_pol = 6, 8, 10, 10, 11, 11, 12`.

**Continuation Method**
 - `pres_ratio` (float): Multiplier on the pressure profile. Default = `1.0`.

 - `bdry_ratio` (float): Multiplier on the 3D boundary modes. Default = `1.0`.

 - `pert_order` (int): Order of the perturbation approximation: `0` = no perturbation, `1` = linear, `2` = quadratic. Default = `1`.

When both pres_ratio = 1 and pres_ratio = 1, the equilibrium is solved using the exact boundary modes and pressure profile as input. pres_ratio = 0 assumes a vacuum pressure profile, and bdry_ratio = 0 ignores all of the non-axisymmetric boundary modes (reducing the input to a tokamak).

These arguments are also passed as arrays for each iteration, with the same notation as the other continuation parameters. This example will start by solving a vacuum tokamak, then perturb the pressure profile to solve a finite-beta tokamak, and finally perturb the boundary to solve the finite-beta stellarator. If only one value is given, as with pert_order in this example, that value will be used for all iterations.

Let's change the default parameters so that we use a 3-step continuation method, where we:

1. First solve a vacuum (`pres_ratio=0`) axisymmetric tokamak (`bdry_ratio=0`), which will use only the axisymmetric modes of the boundary. Let's also choose a poloidal resolution of `M_pol=

We can run this from the DESC input file located in this directory named `SOLOVEV`. From this directory, type in the following command: `desc -vv SOLOVEV -o SOLOVEV_output.h5` (if pip installed DESC) or `python -m desc -vv SOLOVEV -o SOLOVEV_output.h5` (if not pip installed but the DESC directory is on your python path)



Once ran, we can plot the flux surfaces and the force error and see that we indeed have an equilibrium

We could run this from the DESC input file located in `examples/DESC/SOLOVEV`, but to show how DESC can convert from VMEC inputs we will start with the VMEC input file.

### Running DESC on a VMEC Input File

DESC can be run on